In [127]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

In [43]:
f = open('WhatsApp Chat with Say GT (original) 12 hour format.txt', 'r', encoding='utf-8')

In [44]:
data = ' '.join(f.read().split('\n'))

In [45]:
data

'5/9/22, 3:01 PM - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more. 5/26/22, 7:21 PM - Clg Divyansh: Robotics ke notes 5/26/22, 7:44 PM - Clg Kushagra Ahire: MST me jo numerical aaye the uske solution bhej do @918435906042 @919174406481 @918878677076 5/26/22, 7:55 PM - Clg Abbas Amjhera: <Media omitted> 5/26/22, 8:05 PM - +91 6239 413 783: Bhai notes bhej do yrr koi 🙏🏻 5/26/22, 8:05 PM - Clg Kushagra Ahire: mail pe dekh 5/26/22, 8:05 PM - Clg Kushagra Ahire: mam ne bheje hai bahut saare notes 5/26/22, 8:10 PM - +91 6239 413 783: Doosre nhi h acchr 5/26/22, 8:58 PM - +91 96309 40258: Paper hai kya robotics?? 5/26/22, 9:45 PM - Clg Kushagra Ahire: <Media omitted> 5/26/22, 9:45 PM - Clg Kushagra Ahire: <Media omitted> 5/26/22, 9:45 PM - Clg Kushagra Ahire: <Media omitted> 5/26/22, 9:45 PM - Clg Kushagra Ahire: <Media omitted> 5/26/22, 9:45 PM - Clg Kushagra Ahire: <Media omitted> 5/26/22, 9:45 PM - 

## 12 Hour Format

In [35]:
# pattern = "\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s?(?:A|P)\.?M\.?\s-\s"
pattern = "\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s"

In [36]:
messages = re.split(pattern, data)[1:]
messages

['Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n',
 'Clg Divyansh: Robotics ke notes\n',
 'Clg Kushagra Ahire: MST me jo numerical aaye the uske solution bhej do @918435906042 @919174406481 @918878677076\n',
 'Clg Abbas Amjhera: <Media omitted>\n',
 '+91 6239 413 783: Bhai notes bhej do yrr koi 🙏🏻\n',
 'Clg Kushagra Ahire: mail pe dekh\n',
 'Clg Kushagra Ahire: mam ne bheje hai bahut saare notes\n',
 '+91 6239 413 783: Doosre nhi h acchr\n',
 '+91 96309 40258: Paper hai kya robotics??\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Divyansh: Iske lava to kuch nahi hai na\n',
 'Clg Charchil: Mail pr dekh lai ek bar\n',
 'Clg Divyansh: Mail par to pata nahi kya kya hai\n',
 '+91 96309

In [37]:
dates = re.findall(pattern, data)
dates

['5/9/22, 3:01 PM - ',
 '5/26/22, 7:21 PM - ',
 '5/26/22, 7:44 PM - ',
 '5/26/22, 7:55 PM - ',
 '5/26/22, 8:05 PM - ',
 '5/26/22, 8:05 PM - ',
 '5/26/22, 8:05 PM - ',
 '5/26/22, 8:10 PM - ',
 '5/26/22, 8:58 PM - ',
 '5/26/22, 9:45 PM - ',
 '5/26/22, 9:45 PM - ',
 '5/26/22, 9:45 PM - ',
 '5/26/22, 9:45 PM - ',
 '5/26/22, 9:45 PM - ',
 '5/26/22, 9:45 PM - ',
 '5/26/22, 11:42 PM - ',
 '5/26/22, 11:56 PM - ',
 '5/27/22, 8:37 AM - ',
 '5/27/22, 11:46 AM - ',
 '5/27/22, 11:47 AM - ',
 '5/27/22, 12:04 PM - ',
 '5/27/22, 12:04 PM - ',
 '5/27/22, 12:04 PM - ',
 '5/27/22, 12:11 PM - ',
 '5/27/22, 12:13 PM - ',
 '5/27/22, 5:25 PM - ',
 '5/27/22, 5:31 PM - ',
 '5/27/22, 5:35 PM - ',
 '5/27/22, 5:35 PM - ',
 '5/27/22, 5:37 PM - ',
 '5/27/22, 5:38 PM - ',
 '5/28/22, 2:29 AM - ',
 '5/28/22, 3:56 AM - ',
 '5/28/22, 3:56 AM - ',
 '5/28/22, 10:01 AM - ',
 '5/28/22, 6:12 PM - ',
 '5/29/22, 10:52 AM - ',
 '5/29/22, 10:53 AM - ',
 '5/29/22, 10:53 AM - ',
 '5/29/22, 10:54 AM - ',
 '5/29/22, 3:20 PM - ',
 '5

## 24 Hour Format

In [23]:
f = open('WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')

In [24]:
data = f.read()

In [25]:
data

'5/9/22, 15:01 - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n5/26/22, 19:21 - Clg Divyansh: Robotics ke notes\n5/26/22, 19:44 - Clg Kushagra Ahire: MST me jo numerical aaye the uske solution bhej do @918435906042 @919174406481 @918878677076\n5/26/22, 19:55 - Clg Abbas Amjhera: <Media omitted>\n5/26/22, 20:05 - +91 6239 413 783: Bhai notes bhej do yrr koi 🙏🏻\n5/26/22, 20:05 - Clg Kushagra Ahire: mail pe dekh\n5/26/22, 20:05 - Clg Kushagra Ahire: mam ne bheje hai bahut saare notes\n5/26/22, 20:10 - +91 6239 413 783: Doosre nhi h acchr\n5/26/22, 20:58 - +91 96309 40258: Paper hai kya robotics??\n5/26/22, 21:45 - Clg Kushagra Ahire: <Media omitted>\n5/26/22, 21:45 - Clg Kushagra Ahire: <Media omitted>\n5/26/22, 21:45 - Clg Kushagra Ahire: <Media omitted>\n5/26/22, 21:45 - Clg Kushagra Ahire: <Media omitted>\n5/26/22, 21:45 - Clg Kushagra Ahire: <Media omitted>\n5/26/22, 21:45 - Clg Kushagra Ahi

In [26]:
pattern = "\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"

In [27]:
messages = re.split(pattern, data)[1:]
messages

['Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n',
 'Clg Divyansh: Robotics ke notes\n',
 'Clg Kushagra Ahire: MST me jo numerical aaye the uske solution bhej do @918435906042 @919174406481 @918878677076\n',
 'Clg Abbas Amjhera: <Media omitted>\n',
 '+91 6239 413 783: Bhai notes bhej do yrr koi 🙏🏻\n',
 'Clg Kushagra Ahire: mail pe dekh\n',
 'Clg Kushagra Ahire: mam ne bheje hai bahut saare notes\n',
 '+91 6239 413 783: Doosre nhi h acchr\n',
 '+91 96309 40258: Paper hai kya robotics??\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Kushagra Ahire: <Media omitted>\n',
 'Clg Divyansh: Iske lava to kuch nahi hai na\n',
 'Clg Charchil: Mail pr dekh lai ek bar\n',
 'Clg Divyansh: Mail par to pata nahi kya kya hai\n',
 '+91 96309

In [28]:
dates = re.findall(pattern, data)
dates

['5/9/22, 15:01 - ',
 '5/26/22, 19:21 - ',
 '5/26/22, 19:44 - ',
 '5/26/22, 19:55 - ',
 '5/26/22, 20:05 - ',
 '5/26/22, 20:05 - ',
 '5/26/22, 20:05 - ',
 '5/26/22, 20:10 - ',
 '5/26/22, 20:58 - ',
 '5/26/22, 21:45 - ',
 '5/26/22, 21:45 - ',
 '5/26/22, 21:45 - ',
 '5/26/22, 21:45 - ',
 '5/26/22, 21:45 - ',
 '5/26/22, 21:45 - ',
 '5/26/22, 23:42 - ',
 '5/26/22, 23:56 - ',
 '5/27/22, 08:37 - ',
 '5/27/22, 11:46 - ',
 '5/27/22, 11:47 - ',
 '5/27/22, 12:04 - ',
 '5/27/22, 12:04 - ',
 '5/27/22, 12:04 - ',
 '5/27/22, 12:11 - ',
 '5/27/22, 12:13 - ',
 '5/27/22, 17:25 - ',
 '5/27/22, 17:31 - ',
 '5/27/22, 17:35 - ',
 '5/27/22, 17:35 - ',
 '5/27/22, 17:37 - ',
 '5/27/22, 17:38 - ',
 '5/28/22, 02:29 - ',
 '5/28/22, 03:56 - ',
 '5/28/22, 03:56 - ',
 '5/28/22, 10:01 - ',
 '5/28/22, 18:12 - ',
 '5/29/22, 10:52 - ',
 '5/29/22, 10:53 - ',
 '5/29/22, 10:53 - ',
 '5/29/22, 10:54 - ',
 '5/29/22, 15:20 - ',
 '5/29/22, 15:40 - ',
 '5/29/22, 15:57 - ',
 '5/29/22, 16:03 - ',
 '5/30/22, 15:32 - ',
 '5/30/22, 

In [29]:
df = pd.DataFrame({'user_message':messages, 'message_date': dates})
df['message_date'] = pd.to_datetime(df['message_date'], format='%m/%d/%y, %H:%M - ')
df.rename(columns={'message_date':'date'}, inplace=True)
df.head()

,user_message,date
0,Messages and calls are end-to-end encrypted. N...,2022-05-09 15:01:00
1,Clg Divyansh: Robotics ke notes\n,2022-05-26 19:21:00
2,Clg Kushagra Ahire: MST me jo numerical aaye t...,2022-05-26 19:44:00
3,Clg Abbas Amjhera: <Media omitted>\n,2022-05-26 19:55:00
4,+91 6239 413 783: Bhai notes bhej do yrr koi 🙏🏻\n,2022-05-26 20:05:00


In [182]:
def createDf(file_data, dt):
    patterns = {
        '12hr':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s',
        '24hr':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s',
    }
    datetime_formats = {
    '12hr':'%m/%d/%y, %H:%M %p - ',
    '24hr':'%m/%d/%y, %H:%M - ',
    }
    
    data = ' '.join(file_data.read().split('\n'))
    user_message = re.split(patterns[dt], data)[1:]
    date_time = re.findall(patterns[dt], data)
    df = pd.DataFrame({'date_time': date_time, 'user_message': user_message})
    x = str(df['date_time'].iloc[1])
    df['date_time'] = df['date_time'].apply(lambda x: datetime.strptime(x, datetime_formats[dt]).strftime(datetime_formats[dt]))
    
    users = []
    messages = []
    for message in df['user_message']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['user'] = users
    df['message'] = messages
    df.drop(columns=['user_message'], inplace=True)
    return df

In [183]:
f = open('WhatsApp Chat with Say GT (original) 12 hour format.txt', 'r', encoding='utf-8')
createDf(f, '12hr')

,date_time,user,message
0,"05/09/22, 03:01 AM -",group_notification,Messages and calls are end-to-end encrypted. N...
1,"05/26/22, 07:21 AM -",Clg Divyansh,Robotics ke notes
2,"05/26/22, 07:44 AM -",Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...
3,"05/26/22, 07:55 AM -",Clg Abbas Amjhera,<Media omitted>
4,"05/26/22, 08:05 AM -",+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻
...,...,...,...
1452,"08/18/22, 02:01 AM -",Clg Charchil,🤚
1453,"08/18/22, 02:01 AM -",+91 87705 37687,✋🏻
1454,"08/18/22, 02:15 AM -",Clg Atman,🙋‍♂️
1455,"08/18/22, 02:20 AM -",Clg Ashvin birla,<Media omitted>


In [184]:
f = open('WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')
createDf(f, '24hr')

,date_time,user,message
0,"05/09/22, 15:01 -",group_notification,Messages and calls are end-to-end encrypted. N...
1,"05/26/22, 19:21 -",Clg Divyansh,Robotics ke notes
2,"05/26/22, 19:44 -",Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...
3,"05/26/22, 19:55 -",Clg Abbas Amjhera,<Media omitted>
4,"05/26/22, 20:05 -",+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻
...,...,...,...
1437,"08/17/22, 14:13 -",Clg Amit Pandey,ni
1438,"08/17/22, 14:14 -",+91 87705 37687,Nhi
1439,"08/17/22, 14:29 -",Clg Ashvin birla,Hmara to parso paper h
1440,"08/17/22, 14:29 -",Clg Ashvin birla,Subh 9 bje


In [174]:
datetime_formats = {
    '12hr':'%m/%d/%y, %H:%M %p - ',
    '24hr':'%m/%d/%y, %H:%M - ',
}

In [186]:
datetime.strptime('6/1/22, 12:48 AM - ', datetime_formats['12hr']).strftime(datetime_formats['12hr'])

'06/01/22, 12:48 PM - '

In [176]:
x

'5/26/22, 7:21 PM - '

In [177]:
datetime.strptime(x, datetime_formats['12hr']).strftime(datetime_formats['12hr'])

'05/26/22, 07:21 AM - '